In [19]:
from os import listdir
from os.path import isfile, join
import re
demographic_files = [f for f in listdir('BabyNames/') if isfile(join('BabyNames/', f)) and re.search('^yob\d', f)]
demographic_files.sort()

In [21]:
import pandas as pd
def read_file(file_name):
    year = re.search('yob(\d+).txt', file_name).group(1)
    df = pd.read_csv('BabyNames/{0}'.format(file_name), sep=',', header=None)
    df.columns = ['Name', 'Sex', 'Count']
    df['Year'] = year
    return df

read_data_frames = [read_file(f_name) for f_name in demographic_files]

In [23]:
len(read_data_frames)

131

['yob1904.txt', 'yob1990.txt', 'yob1933.txt', 'yob1907.txt', 'yob1963.txt', 'yob1911.txt', 'yob1925.txt', 'yob1882.txt', 'yob1928.txt', 'yob1978.txt', 'yob1951.txt', 'yob1996.txt', 'yob2007.txt', 'yob1929.txt', 'yob1968.txt', 'yob1932.txt', 'yob1955.txt', 'yob1917.txt', 'yob1973.txt', 'yob1991.txt', 'yob1964.txt', 'yob1948.txt', 'yob2006.txt', 'yob2004.txt', 'yob1915.txt', 'yob1967.txt', 'yob1980.txt', 'yob1985.txt', 'yob1969.txt', 'yob1930.txt', 'yob2000.txt', 'yob1982.txt', 'yob1958.txt', 'yob1992.txt', 'yob1941.txt', 'yob1918.txt', 'yob1895.txt', 'yob1952.txt', 'yob1899.txt', 'yob1950.txt', 'yob1883.txt', 'yob1881.txt', 'yob1995.txt', 'yob1943.txt', 'yob1947.txt', 'yob1957.txt', 'yob2005.txt', 'yob1888.txt', 'yob1977.txt', 'yob1999.txt', 'yob1938.txt', 'yob1939.txt', 'yob2002.txt', 'yob1914.txt', 'yob1979.txt', 'yob1913.txt', 'yob1942.txt', 'yob1880.txt', 'yob1954.txt', 'yob1898.txt', 'yob1959.txt', 'yob1919.txt', 'yob1910.txt', 'yob1944.txt', 'yob1988.txt', 'NationalReadMe.pdf', 'y